<a href="https://colab.research.google.com/github/kamilo116/KNN/blob/master/KNN_ascending_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np 
import pandas as pd
import os
import csv
import sys
from random import shuffle
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import imshow
%matplotlib inline
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
from torch.utils.data.sampler import SubsetRandomSampler

import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as T
import torchvision.transforms as transforms
from torchvision import models
import timeit

np.random.seed(4) 
torch.manual_seed(4) 
torch.cuda.manual_seed(4)

In [2]:
from google.colab import drive, files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
! git clone https://github.com/wang-chen/kervolution.git 

sys.path.append("kervolution/")
from kervolution import Kerv2d


fatal: destination path 'kervolution' already exists and is not an empty directory.


In [4]:
MALIGNANT_DATASET = '/content/drive/My Drive/Colab_data/malignant/malignant/'
BENIGN_DATASET = '/content/drive/My Drive/Colab_data/benign/benign/'
DATA_FOLDER = '/content/drive/My Drive/Colab_data/'
LIMIT_IMAGES_NUM = 600

benign_file_list = sorted(os.listdir(BENIGN_DATASET))[:LIMIT_IMAGES_NUM]
malignant_file_list = sorted(os.listdir(MALIGNANT_DATASET))[:LIMIT_IMAGES_NUM]
shuffle(benign_file_list)
shuffle(malignant_file_list)

print(f"Number of benign {len(benign_file_list)} images")
print(f"Number of malignant {len(malignant_file_list)} images")

data_transforms = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])



Number of benign 600 images
Number of malignant 600 images


In [0]:

benign_dict = {filename: 0 for filename in benign_file_list}
malignant_dict = {filename: 1 for filename in malignant_file_list}
img_class_dict = {**benign_dict , **malignant_dict}
labeled_data = pd.Series(img_class_dict)

In [0]:
class IsicDataset(Dataset):
    def __init__(self, data_folder, labeled_data, 
                 transform=transforms.Compose([transforms.ToTensor()])):
        self.labeled_data = labeled_data
        self.transform = transform
        self.data_folder = data_folder
        
        
    def __len__(self):
        return len(self.labeled_data)

    def __getitem__(self, index):
        label = self.labeled_data[index]
        if label == 0:
          image = Image.open(os.path.join(self.data_folder, "benign", "benign", index ))
        else:
          image = Image.open(os.path.join(self.data_folder, "malignant", "malignant", index ))
        image = self.transform(image)
        return image, label

    @property
    def labels(self):
      return self.labeled_data


In [7]:
 
dataset = IsicDataset(DATA_FOLDER, labeled_data, transform=data_transforms)
print(dataset.labels)

X_train, X_test = train_test_split(dataset.labels, test_size=0.2)
print("number of training data: ",len(X_train))
print("number of testing  data: ",len(X_test))

train_sampler = SubsetRandomSampler(list(X_train.index))
valid_sampler = SubsetRandomSampler(list(X_test.index))
batch_size = 64
num_workers = 0

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler, num_workers=num_workers)



ISIC_0000247.jpeg    0
ISIC_0000679.jpeg    0
ISIC_0000007.jpeg    0
ISIC_0000219.jpeg    0
ISIC_0000033.jpeg    0
                    ..
ISIC_0010644.jpeg    1
ISIC_0010207.jpeg    1
ISIC_0000167.jpeg    1
ISIC_0001127.jpeg    1
ISIC_0000521.jpeg    1
Length: 1200, dtype: int64
number of training data:  960
number of testing  data:  240


In [0]:
avg_loss_list = []
acc_list = []

def train(model, train_loader ,loss_fn, optimizer, num_epochs = 1):
    total_loss =0

    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        model.train()

        for t, (x, y) in enumerate(train_loader):
            x_var = Variable(x.type(gpu_dtype))
            y_var = Variable(y.type(gpu_dtype).long())
            scores = model(x_var)
            loss = loss_fn(scores, y_var)
            total_loss += loss.data
            
            if (t + 1) % print_every == 0:
                avg_loss = total_loss/print_every
                print('t = %d, avg_loss = %.4f' % (t + 1, avg_loss) )
                avg_loss_list.append(avg_loss)
                total_loss = 0
                

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        acc = check_accuracy(fixed_model_gpu, valid_loader)
        print('acc = %f' %(acc))
            
def check_accuracy(model, loader):
    print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval() 
    for x, y in loader:
        x_var = Variable(x.type(gpu_dtype))

        scores = model(x_var)
        _, preds = scores.data.cpu().max(1)
        num_correct += (preds == y).sum()
        num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    acc_list.append(acc)
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
    return acc
    

In [0]:
class Flatten(nn.Module):
    def forward(self, x):
        N, C, H, W = x.size()
        return x.view(N, -1)  

In [10]:


print_every = 1
gpu_dtype = torch.cuda.FloatTensor

out_1 = 32
out_2 = 64
out_3 = 128
#out_4 = 256


k_size_1 = 3
padding_1 = 1
in_channels = 3

num_epochs = 50

'''
Kerv2d
kervolution with following options:
kernel_type: [linear, polynomial, gaussian, etc.]
default is convolution:
          kernel_type --> linear,
balance, power, gamma is valid only when the kernel_type is specified
if learnable_kernel = True,  they just be the initial value of learable parameters
if learnable_kernel = False, they are the value of kernel_type's parameter
the parameter [power] cannot be learned due to integer limitation
dilation (int or tuple, optional): Spacing between kernel
elements. Default: 1
groups (int, optional): Number of blocked connections from input
channels to output channels. Default: 1
bias (bool, optional): If ``True``, adds a learnable bias to the output. Default: ``True``
kernel_type (str), Default: 'linear'
learnable_kernel (bool): Learnable kernel parameters.  Default: False 
balance: 0, 1
power: 3, 4, 5
gamma:
'''


fixed_model_base = nn.Sequential( 
                nn.Kerv2d(in_channels , out_1, padding=padding_1, dilation=1, groups=1, bias=True, 
                          kernel_type='polynomial', kernel_size=k_size_1, learnable_kernel=True,
                          kernel_regularizer=True, stride=1, balance=0, power=3, gamma=1), 
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(out_1),
                nn.Conv2d(out_1 , out_2, padding= padding_1, kernel_size=k_size_1, stride=1), 
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(out_2),
                nn.MaxPool2d(2, stride=2),
                nn.Conv2d(out_2 , out_3, padding= padding_1, kernel_size=k_size_1, stride=1),  
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(out_3),
                nn.MaxPool2d(2, stride=2),
                nn.Dropout(0.5),
                Flatten(),
                nn.Linear(80000,64),
                nn.ReLU(inplace=True),
                nn.Linear(64,2)
            )
fixed_model_gpu = fixed_model_base.type(gpu_dtype)
print(fixed_model_gpu)
loss_fn = nn.modules.loss.CrossEntropyLoss()
optimizer = optim.Adam(fixed_model_gpu.parameters(), lr = 0.0001, weight_decay=0) 

train(fixed_model_gpu, train_loader ,loss_fn, optimizer, num_epochs=num_epochs)
check_accuracy(fixed_model_gpu, valid_loader)

Sequential(
  (0): Kerv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): ReLU(inplace=True)
  (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (11): Dropout(p=0.5, inplace=False)
  (12): Flatten()
  (13): Linear(in_features=80000, out_features=64, bias=True)
  (14): ReLU(inplace=True)
  (15): Linear(in_features=64, out_features=2, bias=True)
)
Starting epoch 1 / 50
t = 1, avg_loss = 0.6603
t = 2, avg_l

KeyboardInterrupt: ignored

In [0]:
import matplotlib.pyplot as plt

plt.plot([print_every*batch_size*(i+1) for i in range((len(avg_loss_list)))],avg_loss_list)
print("Loss:")
plt.show()

In [0]:
plt.plot([i+1 for i in range((len(acc_list)))],acc_list)
print("Accurancy:")
plt.show()